In [1]:
import sqlite3
import pandas as pd
from tabulate import tabulate

Step 1: Function to read database and list table schemas and record counts

In [2]:
def analyze_database(db_path):
    """
    Connects to the SQLite database, retrieves table schemas, and counts records.
    :param db_path: Path to the SQLite database file.
    :return: None (Prints information in table format).
    """
    try:
        conn = sqlite3.connect(db_path)
        cursor = conn.cursor()

        # Fetch all table names
        cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
        tables = cursor.fetchall()

        print("Tables in the database:")
        table_info = []

        for table in tables:
            table_name = table[0]

            # Get schema
            cursor.execute(f"PRAGMA table_info({table_name});")
            schema = cursor.fetchall()
            schema_info = [(column[1], column[2]) for column in schema]

            # Count records
            cursor.execute(f"SELECT COUNT(*) FROM {table_name};")
            record_count = cursor.fetchone()[0]

            table_info.append([table_name, record_count, schema_info])

        # Print table info as a formatted table
        headers = ["Table Name", "Record Count", "Schema"]
        print(tabulate(table_info, headers=headers, tablefmt="grid"))

    except sqlite3.Error as e:
        print(f"Database error: {e}")
    finally:
        if conn:
            conn.close()

In [3]:
database_path = 'database.db'
analyze_database(database_path)

Tables in the database:
+-----------------+----------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| Table Name      |   Record Count | Schema                                                                                                                                                                                                                                                                                                                                                                                                                           |
+=================+================+============================

Step 2: Query answers based on ERD and provided questions

In [4]:
# Determine schema type based on ERD
def question_a():
    print("\nQuestion a: Database Schema")
    print("Based on the ERD, this database represents a Snowflake schema because it normalizes data across multiple related tables.")

In [5]:
question_a()


Question a: Database Schema
Based on the ERD, this database represents a Snowflake schema because it normalizes data across multiple related tables.


In [6]:
# Number of different customers
def question_b(conn):
    print("\nQuestion b: Number of Customers")
    query = "SELECT COUNT(DISTINCT CustomerId) AS customer_count FROM customers;"
    print("SQL Query:", query)
    customer_count = pd.read_sql_query(query, conn)
    print("Result:")
    print(customer_count)
    
    # Pandas Equivalent
    df_customers = pd.read_sql_query("SELECT * FROM customers;", conn)
    print("Pandas Equivalent Result:")
    print(df_customers['CustomerId'].nunique())

In [7]:
conn = sqlite3.connect(database_path)

In [8]:
question_b(conn)


Question b: Number of Customers
SQL Query: SELECT COUNT(DISTINCT CustomerId) AS customer_count FROM customers;
Result:
   customer_count
0              59
Pandas Equivalent Result:
59


In [9]:
def question_c(conn):
    print("\nQuestion c: Genre with Shortest Average Track Length")
    # SQL Query
    query = """
        SELECT g.Name AS Genre, AVG(t.Milliseconds / 60000.0) AS AvgLengthMinutes
        FROM tracks t
        JOIN genres g ON t.GenreId = g.GenreId
        GROUP BY g.Name
        ORDER BY AvgLengthMinutes ASC
        LIMIT 1;
    """
    print("SQL Query:")
    print(query)
    genre_result = pd.read_sql_query(query, conn)
    print("\nSQL Result:")
    print(genre_result)

    # Pandas Equivalent
    print("\nPandas Equivalent:")
    df_tracks = pd.read_sql_query("SELECT * FROM tracks;", conn)
    df_genres = pd.read_sql_query("SELECT * FROM genres;", conn)

    # Ensure genre name column is correctly identified
    print("Genres Table Columns:", df_genres.columns)
    genre_name_column = 'Name' if 'Name' in df_genres.columns else df_genres.columns[-1]

    df_tracks['LengthMinutes'] = df_tracks['Milliseconds'] / 60000.0
    merged = df_tracks.merge(df_genres, left_on='GenreId', right_on='GenreId')
    print("Merged Table Columns:", merged.columns)

    # Identify correct column in merged DataFrame
    genre_column = 'Name_y' if 'Name_y' in merged.columns else 'Name_x'

    genre_avg_lengths = merged.groupby(genre_column)['LengthMinutes'].mean()
    shortest_genre = genre_avg_lengths.idxmin()
    shortest_length = genre_avg_lengths.min()
    print(f"Genre: {shortest_genre}, Avg Length (Minutes): {shortest_length}")

In [10]:
question_c(conn)


Question c: Genre with Shortest Average Track Length
SQL Query:

        SELECT g.Name AS Genre, AVG(t.Milliseconds / 60000.0) AS AvgLengthMinutes
        FROM tracks t
        JOIN genres g ON t.GenreId = g.GenreId
        GROUP BY g.Name
        ORDER BY AvgLengthMinutes ASC
        LIMIT 1;
    

SQL Result:
           Genre  AvgLengthMinutes
0  Rock And Roll          2.244058

Pandas Equivalent:
Genres Table Columns: Index(['GenreId', 'Name'], dtype='object')
Merged Table Columns: Index(['TrackId', 'Name_x', 'AlbumId', 'MediaTypeId', 'GenreId', 'Composer',
       'Milliseconds', 'Bytes', 'UnitPrice', 'LengthMinutes', 'Name_y'],
      dtype='object')
Genre: Rock And Roll, Avg Length (Minutes): 2.2440583333333333


In [11]:
# Artist appearing most in playlists
def question_d(conn):
    print("\nQuestion d: Artist Most in Playlists")
    # SQL Query
    query = """
        SELECT a.Name AS Artist, COUNT(pl.TrackId) AS Occurrences
        FROM playlist_track pl
        JOIN tracks t ON pl.TrackId = t.TrackId
        JOIN albums al ON t.AlbumId = al.AlbumId
        JOIN artists a ON al.ArtistId = a.ArtistId
        GROUP BY a.Name
        ORDER BY Occurrences DESC
        LIMIT 1;
    """
    print("SQL Query:", query)
    artist_result = pd.read_sql_query(query, conn)
    print("SQL Result:")
    print(artist_result)

    # Pandas Equivalent
    print("\nPandas Equivalent:")
    df_playlist_track = pd.read_sql_query("SELECT * FROM playlist_track;", conn)
    df_tracks = pd.read_sql_query("SELECT * FROM tracks;", conn)
    df_albums = pd.read_sql_query("SELECT * FROM albums;", conn)
    df_artists = pd.read_sql_query("SELECT * FROM artists;", conn)

    # Merge data step by step
    merged = df_playlist_track.merge(df_tracks, left_on='TrackId', right_on='TrackId')
    merged = merged.merge(df_albums, left_on='AlbumId', right_on='AlbumId')
    merged = merged.merge(df_artists, left_on='ArtistId', right_on='ArtistId')

    # Verify column names
    print("Merged Table Columns:", merged.columns)

    # Dynamically identify the column for artist name
    artist_column = 'Name' if 'Name' in merged.columns else merged.columns[-1]

    # Calculate most played artist
    artist_most_played = merged[artist_column].value_counts().idxmax()
    most_played_count = merged[artist_column].value_counts().max()
    print(f"Artist: {artist_most_played}, Occurrences: {most_played_count}")

In [12]:
question_d(conn)


Question d: Artist Most in Playlists
SQL Query: 
        SELECT a.Name AS Artist, COUNT(pl.TrackId) AS Occurrences
        FROM playlist_track pl
        JOIN tracks t ON pl.TrackId = t.TrackId
        JOIN albums al ON t.AlbumId = al.AlbumId
        JOIN artists a ON al.ArtistId = a.ArtistId
        GROUP BY a.Name
        ORDER BY Occurrences DESC
        LIMIT 1;
    
SQL Result:
        Artist  Occurrences
0  Iron Maiden          516

Pandas Equivalent:
Merged Table Columns: Index(['PlaylistId', 'TrackId', 'Name_x', 'AlbumId', 'MediaTypeId', 'GenreId',
       'Composer', 'Milliseconds', 'Bytes', 'UnitPrice', 'Title', 'ArtistId',
       'Name_y'],
      dtype='object')
Artist: Iron Maiden, Occurrences: 516


In [13]:
# Customers who spent more than $40
def question_e(conn):
    print("\nQuestion e: Customers Spending > $40")
    query = """
        SELECT COUNT(*) AS HighSpenders
        FROM (
            SELECT c.CustomerId, SUM(il.UnitPrice * il.Quantity) AS TotalSpent
            FROM customers c
            JOIN invoices i ON c.CustomerId = i.CustomerId
            JOIN invoice_items il ON i.InvoiceId = il.InvoiceId
            GROUP BY c.CustomerId
            HAVING TotalSpent > 40
        ) subquery;
    """
    print("SQL Query:", query)
    spenders_result = pd.read_sql_query(query, conn)
    print("Result:")
    print(spenders_result)
    
    # Pandas Equivalent
    df_invoices = pd.read_sql_query("SELECT * FROM invoices;", conn)
    df_invoice_items = pd.read_sql_query("SELECT * FROM invoice_items;", conn)
    merged = df_invoices.merge(df_invoice_items, left_on='InvoiceId', right_on='InvoiceId')

    # Calculate total spent per customer
    merged['TotalSpent'] = merged['UnitPrice'] * merged['Quantity']
    total_spent = merged.groupby('CustomerId')['TotalSpent'].sum()
    high_spenders = (total_spent > 40).sum()

    # Print results
    print(tabulate([[high_spenders]], headers=["High Spenders"], tablefmt="grid"))

In [14]:
question_e(conn)


Question e: Customers Spending > $40
SQL Query: 
        SELECT COUNT(*) AS HighSpenders
        FROM (
            SELECT c.CustomerId, SUM(il.UnitPrice * il.Quantity) AS TotalSpent
            FROM customers c
            JOIN invoices i ON c.CustomerId = i.CustomerId
            JOIN invoice_items il ON i.InvoiceId = il.InvoiceId
            GROUP BY c.CustomerId
            HAVING TotalSpent > 40
        ) subquery;
    
Result:
   HighSpenders
0            14
+-----------------+
|   High Spenders |
+=================+
|              14 |
+-----------------+


In [15]:
# Most frequently used key
def question_f(conn):
    response = """
    Answer to the Question: "Which Key did you use the most?"
    
    From the queries written:
    
    - Primary Key Usage:
      - CustomerId was heavily used in the customers table for filtering and grouping.
      - TrackId was frequently joined across tables (tracks, playlist_track, etc.).
    
    - Foreign Key Usage:
      - Foreign keys like GenreId, AlbumId, ArtistId, and InvoiceId were also commonly used for joining tables.
    
    Response:
    If the key usage refers to the queries: The key used the most in the queries is TrackId.
    This key was utilized extensively across multiple tables (tracks, playlist_track, etc.) to join data for generating insights, particularly in determining:
    
    - The artist appearing most in playlists.
    - The genre with the shortest average track length.
    """
    print(response)

In [16]:
question_f(conn)


    Answer to the Question: "Which Key did you use the most?"
    
    From the queries written:
    
    - Primary Key Usage:
      - CustomerId was heavily used in the customers table for filtering and grouping.
      - TrackId was frequently joined across tables (tracks, playlist_track, etc.).
    
    - Foreign Key Usage:
      - Foreign keys like GenreId, AlbumId, ArtistId, and InvoiceId were also commonly used for joining tables.
    
    Response:
    If the key usage refers to the queries: The key used the most in the queries is TrackId.
    This key was utilized extensively across multiple tables (tracks, playlist_track, etc.) to join data for generating insights, particularly in determining:
    
    - The artist appearing most in playlists.
    - The genre with the shortest average track length.
    


In [17]:
conn.close()